In [4]:
import pandas as pd
import numpy as np
import openml
from sklearn.linear_model import LinearRegression 
import lightgbm as lgbm
import optuna
from scipy.spatial.distance import mahalanobis
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from engression import engression, engression_bagged
import torch

In [2]:
#openml.config.apikey = 'FILL_IN_OPENML_API_KEY'  # set the OpenML Api Key
SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite

task = openml.tasks.get_task(361072)  # download the OpenML task
dataset = task.get_dataset()

X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)

c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [3]:
# calculate the mean and covariance matrix of the dataset
mean = np.mean(X, axis=0)
cov = np.cov(X.T)

# calculate the Mahalanobis distance for each data point
mahalanobis_dist = [mahalanobis(x, mean, np.linalg.inv(cov)) for x in X.values]

mahalanobis_dist=pd.Series(mahalanobis_dist,index=X.index)
far_index=mahalanobis_dist.index[np.where(mahalanobis_dist>=np.quantile(mahalanobis_dist,0.8))[0]]
close_index=mahalanobis_dist.index[np.where(mahalanobis_dist<np.quantile(mahalanobis_dist,0.8))[0]]

X_train = X.loc[close_index,:]
X_test = X.loc[far_index,:]
y_train = y.loc[close_index]
y_test = y.loc[far_index]

mean = np.mean(X_train, axis=0)
cov = np.cov(X_train.T)

# calculate the Mahalanobis distance for each data point
mahalanobis_dist_ = [mahalanobis(x, mean, np.linalg.inv(cov)) for x in X_train.values]

mahalanobis_dist_=pd.Series(mahalanobis_dist_,index=X_train.index)
far_index_=mahalanobis_dist_.index[np.where(mahalanobis_dist_>=np.quantile(mahalanobis_dist_,0.8))[0]]
close_index_=mahalanobis_dist_.index[np.where(mahalanobis_dist_<np.quantile(mahalanobis_dist_,0.8))[0]]

X_train_ = X_train.loc[close_index_,:]
X_val = X_train.loc[far_index_,:]
y_train_ = y_train.loc[close_index_]
y_val = y_train.loc[far_index_]

In [8]:
N_TRIALS=5

def gp(trial):

    params = {'lenghtscale': trial.suggest_float('lenghtscale', 0, 10)}
    
    gp=GaussianProcessRegressor(kernel=Matern(length_scale=params['lenghtscale'], nu=1.5))
    gp.fit(X_train_, y_train_)
    y_val_hat_gp=gp.predict(X_val)
    RMSE_gp=np.sqrt(np.mean((y_val-y_val_hat_gp)**2))
    return RMSE_gp

sampler_gp = optuna.samplers.TPESampler(seed=10)
study_gp = optuna.create_study(sampler=sampler_gp, direction='minimize')
study_gp.optimize(gp, n_trials=N_TRIALS)

gp_model=GaussianProcessRegressor(kernel=Matern(length_scale=study_gp.best_params['lenghtscale'], nu=1.5))


def boosted(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5),
              'n_estimators': trial.suggest_int('n_estimators', 100, 500),
              'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
              'max_depth': trial.suggest_int('max_depth', 1, 30),
              #'num_leaves': trial.suggest_int('num_leaves', 2, 100),
              'min_child_samples': trial.suggest_int('min_child_samples', 10, 100)}
    
    boosted_tree_model=lgbm.LGBMRegressor(**params)
    boosted_tree_model.fit(X_train_, y_train_)
    y_val_hat_boost=boosted_tree_model.predict(X_val)
    RMSE_boost=np.sqrt(np.mean((y_val-y_val_hat_boost)**2))

    return RMSE_boost

sampler_boost = optuna.samplers.TPESampler(seed=10)
study_boost = optuna.create_study(sampler=sampler_boost, direction='minimize')
study_boost.optimize(boosted, n_trials=N_TRIALS)

boosted_model=lgbm.LGBMRegressor(**study_boost.best_params)


def rf(trial):

    params = {'boosting_type':"rf",
              'n_estimators': trial.suggest_int('n_estimators', 100, 500),
              'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
              'max_depth': trial.suggest_int('max_depth', 1, 30),
              'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
              'subsample': trial.suggest_float('subsample', 0.5, 1),
              'feature_fraction_bynode': trial.suggest_float('feature_fraction_bynode', 0, 1),
              #'num_leaves': trial.suggest_int('num_leaves', 2, 100),
              'min_child_samples': trial.suggest_int('min_child_samples', 10, 100)}
    
    rf_model=lgbm.LGBMRegressor(**params)
    rf_model.fit(X_train_, y_train_)
    y_val_hat_rf=rf_model.predict(X_val)
    RMSE_rf=np.sqrt(np.mean((y_val-y_val_hat_rf)**2))

    return RMSE_rf

sampler_rf = optuna.samplers.TPESampler(seed=10)
study_rf = optuna.create_study(sampler=sampler_rf, direction='minimize')
study_rf.optimize(rf, n_trials=N_TRIALS)

rf_model=lgbm.LGBMRegressor(**study_rf.best_params)

def engressor_NN(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5),
              'num_epoches': trial.suggest_int('num_epoches', 100, 1000),
              'num_layer': trial.suggest_int('num_layer', 2, 5),
              'hidden_dim': trial.suggest_int('hidden_dim', 50, 100),
              'noise_dim': trial.suggest_int('noise_dim', 50, 100),}
    
    engressor_model=engression(torch.Tensor(np.array(X_train_)), torch.Tensor(np.array(y_train_).reshape(-1,1)), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=1000)
    y_val_hat_engression=engressor_model.predict(torch.Tensor(np.array(X_val)), target="mean")
    RMSE_engression=np.sqrt((((torch.Tensor(np.array(y_val).reshape(-1,1)))-y_val_hat_engression)**2).mean(axis=0))

    return RMSE_engression

sampler_engression = optuna.samplers.TPESampler(seed=10)
study_engression = optuna.create_study(sampler=sampler_engression, direction='minimize')
study_engression.optimize(engressor_NN, n_trials=N_TRIALS)

[I 2023-12-04 16:52:36,341] A new study created in memory with name: no-name-76f857dd-846f-45c7-a01d-187f2ac6312b
[I 2023-12-04 16:54:07,410] Trial 0 finished with value: 36.3988536368184 and parameters: {'lenghtscale': 7.71320643266746}. Best is trial 0 with value: 36.3988536368184.
[I 2023-12-04 16:54:17,239] Trial 1 finished with value: 79.97454305109773 and parameters: {'lenghtscale': 0.207519493594015}. Best is trial 0 with value: 36.3988536368184.
[I 2023-12-04 16:55:48,661] Trial 2 finished with value: 36.398853635730084 and parameters: {'lenghtscale': 6.336482349262754}. Best is trial 2 with value: 36.398853635730084.
[I 2023-12-04 16:58:50,774] Trial 3 finished with value: 36.398854693719755 and parameters: {'lenghtscale': 7.488038825386118}. Best is trial 2 with value: 36.398853635730084.
[I 2023-12-04 17:02:23,089] Trial 4 finished with value: 36.39890062205961 and parameters: {'lenghtscale': 4.9850701230259045}. Best is trial 2 with value: 36.398853635730084.
[I 2023-12-04 

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001871 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3766
[LightGBM] [Info] Number of data points in the train set: 5242, number of used features: 21
[LightGBM] [Info] Start training from score 91.111980
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


[I 2023-12-04 17:02:23,549] Trial 1 finished with value: 22.810531505875442 and parameters: {'learning_rate': 0.11317352611989298, 'n_estimators': 179, 'reg_lambda': 0.0699481785242808, 'max_depth': 6, 'min_child_samples': 18}. Best is trial 1 with value: 22.810531505875442.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000399 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3766
[LightGBM] [Info] Number of data points in the train set: 5242, number of used features: 21
[LightGBM] [Info] Start training from score 91.111980
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

[I 2023-12-04 17:02:24,143] Trial 2 finished with value: 22.85055837450557 and parameters: {'learning_rate': 0.3429945493655308, 'n_estimators': 482, 'reg_lambda': 1.08526150100961e-08, 'max_depth': 16, 'min_child_samples': 83}. Best is trial 1 with value: 22.810531505875442.


...

[I 2023-12-04 17:04:35,226] Trial 2 finished with value: 127.4811782836914 and parameters: {'learning_rate': 0.3429945493655308, 'num_epoches': 959, 'num_layer': 2, 'hidden_dim': 76, 'noise_dim': 91}. Best is trial 1 with value: 13.67954158782959.



Training loss on the original (non-standardized) scale:
	Energy-loss: 1.2577,  E(|Y-Yhat|): 2.6051,  E(|Yhat-Yhat'|): 2.6949

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a mismatch 
Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1000.
[Epoch 1 (0%), batch 6] energy-loss: 0.7243,  E(|Y-Yhat|): 1.5722,  E(|Yhat-Yhat'|): 1.6958
[Epoch 100 (13%), batch 6] energy-loss: 0.2150,  E(|Y-Yhat|): 0.4371,  E(|Yhat-Yhat'|): 0.4441
[Epoch 200 (27%), batch 6] energy-loss: 0.2290,  E(|Y-Yhat|): 0.4668,  E(|Yhat-Yhat'|): 0.4756
[Epoch 300 (40%), batch 6] energy-loss: 0.2033,  E(|Y-Yhat|): 0.4286,  E(|Yhat-Yhat'|): 0.4506
[Epoch 400 (53%)

[I 2023-12-04 17:05:57,508] Trial 3 finished with value: 170.4100799560547 and parameters: {'learning_rate': 0.3066505073478647, 'num_epoches': 750, 'num_layer': 3, 'hidden_dim': 96, 'noise_dim': 86}. Best is trial 1 with value: 13.67954158782959.


Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1000.
[Epoch 1 (0%), batch 6] energy-loss: 0.8017,  E(|Y-Yhat|): 1.2023,  E(|Yhat-Yhat'|): 0.8012
[Epoch 100 (43%), batch 6] energy-loss: 0.2099,  E(|Y-Yhat|): 0.4407,  E(|Yhat-Yhat'|): 0.4616
[Epoch 200 (87%), batch 6] energy-loss: 0.2332,  E(|Y-Yhat|): 0.4366,  E(|Yhat-Yhat'|): 0.4068

Training loss on the original (non-standardized) scale:
	Energy-loss: 1.0694,  E(|Y-Yhat|): 2.2098,  E(|Yhat-Yhat'|): 2.2809

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a mismatch 


[I 2023-12-04 17:06:21,196] Trial 4 finished with value: 30.25997543334961 and parameters: {'learning_rate': 0.27172963963761937, 'num_epoches': 228, 'num_layer': 3, 'hidden_dim': 84, 'noise_dim': 72}. Best is trial 1 with value: 13.67954158782959.


In [9]:
gp_model.fit(X_train, y_train)
y_test_hat_gp=gp_model.predict(X_test)
RMSE_gp=np.sqrt(np.mean((y_test-y_test_hat_gp)**2))

boosted_model.fit(X_train, y_train)
y_test_hat_boosted=boosted_model.predict(X_test)
RMSE_boosted=np.sqrt(np.mean((y_test-y_test_hat_boosted)**2))

rf_model.fit(X_train, y_train)
y_test_hat_rf=rf_model.predict(X_test)
RMSE_rf=np.sqrt(np.mean((y_test-y_test_hat_rf)**2))

lin_reg=LinearRegression()
lin_reg.fit(X_train, y_train)
y_test_hat_linreg=lin_reg.predict(X_test)
RMSE_linreg=np.sqrt(np.mean((y_test-y_test_hat_linreg)**2))

params=study_engression.best_params
engressor_model=engression(torch.Tensor(np.array(X_train)), torch.Tensor(np.array(y_train).reshape(-1,1)), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=1000)
y_test_hat_engression=engressor_model.predict(torch.Tensor(np.array(X_test)), target="mean")
RMSE_engression=np.sqrt((((torch.Tensor(np.array(y_test).reshape(-1,1)))-y_test_hat_engression)**2).mean(axis=0))

print("RMSE linear regression: ",RMSE_linreg)
print("RMSE boosted trees", RMSE_boosted)
print("RMSE random forest", RMSE_rf)
print("RMSE gaussian process", RMSE_gp)
print("RMSE engression", RMSE_engression)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002056 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4351
[LightGBM] [Info] Number of data points in the train set: 6553, number of used features: 21
[LightGBM] [Info] Start training from score 88.320464
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

RuntimeError: The size of tensor a (1311) must match the size of tensor b (1639) at non-singleton dimension 0

In [10]:
RMSE_engression=np.sqrt((((torch.Tensor(np.array(y_test).reshape(-1,1)))-y_test_hat_engression)**2).mean(axis=0))

print("RMSE linear regression: ",RMSE_linreg)
print("RMSE boosted trees", RMSE_boosted)
print("RMSE random forest", RMSE_rf)
print("RMSE gaussian process", RMSE_gp)
print("RMSE engression", RMSE_engression)

RMSE linear regression:  62.070783717577555
RMSE boosted trees 5.276375538454642
RMSE random forest 5.220881219934237
RMSE gaussian process 38.12484784299995
RMSE engression tensor([34.8711])
